## Quick Start: Training an IMDb sentiment model with *ULMFiT*

Let's start with a quick end-to-end example of training a model. We'll train a sentiment classifier on a sample of the popular IMDb data, showing 4 steps:

1. Reading and viewing the IMDb data
1. Getting your data ready for modeling
1. Fine-tuning a language model
1. Building a classifier

In [1]:
from fastai.text import * 

In [2]:
torch.cuda.set_device(1)

Contrary to images in Computer Vision, text can't directly be transformed into numbers to be fed into a model. The first thing we need to do is to preprocess our data so that we change the raw texts to lists of words, or tokens (a step that is called tokenization) then transform these tokens into numbers (a step that is called numericalization). These numbers are then passed to embedding layers that will convert them in arrays of floats before passing them through a model.

Steps:

1. Get your data preprocessed and ready to use,
1. Create a language model with pretrained weights that you can fine-tune to your dataset,
1. Create other models such as classifiers on top of the encoder of the language model.

To show examples, we have provided a small sample of the [IMDB dataset](https://www.imdb.com/interfaces/) which contains 1,000 reviews of movies with labels (positive or negative).

In [3]:
path = untar_data(URLs.IMDB_SAMPLE)

In [4]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


#### Get the data and create a `databunch` object to use for a language model

In [5]:
%%time

# throws `BrokenProcessPool` Error sometimes. Keep trying `till it works!
count = 0
error = True
while error:
    try: 
        # The following line throws `AttributeError: backwards` on the learning step, below
        # data_lm = TextDataBunch.from_csv(path, 'texts.csv')
        # This Fastai Forum post shows the solution:
        #      https://forums.fast.ai/t/backwards-attributes-not-found-in-nlp-text-learner/51340?u=jcatanza
        # We implement the solution on the following line:
        data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')
        error = False
        print(f'failure count is {count}\n')    
    except: # catch *all* exceptions
        # accumulate failure count
        count = count + 1
        print(f'failure count is {count}')

/home/molly/miniconda3/envs/fastai1-8/lib/python3.8/site-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


failure count is 0

CPU times: user 257 ms, sys: 197 ms, total: 454 ms
Wall time: 1.74 s


#### Get the data again, this time form a `databunch` object for use in a classifier model

In [6]:
%%time

# throws `BrokenProcessPool` Error sometimes. Keep trying `till it works!
count = 0
error = True
while error:
    try: 
        # Create the databunch for the classifier model
        data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=32)
        error = False
        print(f'failure count is {count}\n')    
    except: # catch *all* exceptions
        # accumulate failure count
        count = count + 1
        print(f'failure count is {count}')


failure count is 0

CPU times: user 270 ms, sys: 182 ms, total: 452 ms
Wall time: 1.71 s


/home/molly/miniconda3/envs/fastai1-8/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


#### Save the `databunch` objects for later use

In [7]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

#### Load the `databunch` objects
Note that you can load the data with different [`DataBunch`](/basic_data.html#DataBunch) parameters (batch size, `bptt`,...)

In [8]:
#bs=192
bs=48
#bs=32

In [9]:
data_lm = load_data(path, 'data_lm_export.pkl', bs=bs)
data_clas = load_data(path, 'data_clas_export.pkl', bs=bs)

/home/molly/miniconda3/envs/fastai1-8/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


### 1. Build and fine-tune a language model

We can use the `data_lm` object we created earlier to fine-tune a pretrained language model. [fast.ai](http://www.fast.ai/) has an English model with an AWD-LSTM architecture available that we can download. We can create a learner object that will directly create a model, download the pretrained weights and be ready for fine-tuning.

#### Set up to use the GPU

In [10]:
torch.cuda.set_device(1)
#torch.cuda.set_device(0)

#### Build the IMDb language model, initializing with the hpretrained weights from `wikitext-103`

In [11]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

#### Start training
By default this step unfreezes and trains the weights for the last layer. Training updates the weights to values more applicable to the language of IMDb reviews.

In [12]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.312819,3.903156,0.287798,00:07


You can use [Visual Studio Code](https://code.visualstudio.com/) (vscode - open source editor that comes with recent versions of Anaconda, or can be installed separately), or most editors and IDEs, to browse code. vscode things to know:

- Command palette (<kbd>Ctrl-shift-p</kbd>)
- Go to symbol (<kbd>Ctrl-t</kbd>)
- Find references (<kbd>Shift-F12</kbd>)
- Go to definition (<kbd>F12</kbd>)
- Go back (<kbd>alt-left</kbd>)
- View documentation
- Hide sidebar (<kbd>Ctrl-b</kbd>)
- Zen mode (<kbd>Ctrl-k,z</kbd>)

#### Unfreeze the weights and train some more

Like a computer vision model, we can then unfreeze the model and fine-tune it.
In this step we are allowing the model to update *all* the weights with values more suitable to the language of IMDb reviews. But we are mindful that the pretrained weights from wikitext-103 are likely already near their optimal values. For this we train the weights in the "earlier" layers with a much lower learning rate.

In [13]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4,1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,3.978377,3.887536,0.287672,00:09
1,3.717002,3.865872,0.289583,00:09
2,3.266967,3.886790,0.287970,00:09


#### Test the language model

To evaluate your language model, you can run the [`Learner.predict`](/basic_train.html#Learner.predict) method and specify the number of words you want it to guess.

In [14]:
learn.predict("This is a review about", n_words=10)

'This is a review about the realized european of Africa , which he supposedly'

In [15]:
learn.predict("In the hierarchy of horror movies this has to be near the top.", n_words=10)

"In the hierarchy of horror movies this has to be near the top. It 's a bullets attempt at action ."

Sometimes the generated text doesn't make much sense because we have a tiny vocabulary and didn't train much. But note that the model respects basic grammar, which comes from the pretrained model.

Finally we save the encoder to be able to use it for classification in the next section.

#### Save the encoder

In [16]:
learn.save('mini_imdb_language_model')
learn.save_encoder('mini_imdb_language_model_encoder')

### 2. Build a movie review classifier
We use mixed precision (`.to_fp16()`)for greater speed, smaller memory footprint, and a regularizing effect.

In [17]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5).to_fp16()

In [18]:
learn.load_encoder('mini_imdb_language_model_encoder')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (799 items)
x: TextList
xxbos xxmaj this is a bad b movie xxunk as a mockumentary . xxmaj the porn documentary filmmaker in the movie has almost as much screen time and dialog as any other character . xxmaj that completely destroyed any " documentary feel " that they may have wanted to create . 
 
  xxmaj the fact that the film is not actually a mockumentary is the least of it 's problems . xxmaj the film is not funny . xxmaj the film is not sexy . xxmaj the film does n't have anything xxunk to say about the porn business . xxmaj it 's not even particularly xxunk . xxmaj while there 's xxunk sex , the amount of nudity is mild for an r rated film . 
 
  xxmaj someday , someone will make a good mockumentary about the porn industry . xxmaj this is not it .,xxbos xxmaj the first half was xxup ok , but the last half really , really disappointed . xxmaj it 's funny the producers even admitted they did n't have a clue for the ending , and i

In [19]:
data_clas.show_batch()

/home/molly/miniconda3/envs/fastai1-8/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , xxunk bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the sweetest and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj this film sat on my xxmaj tivo for weeks before i watched it . i dreaded a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the screwed - up xxunk of xxmaj new xxmaj xxunk . \n \n xxmaj the format is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj ronde",positive
"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first xxunk games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - rounded gaming experience in general . xxmaj with graphics",positive
"xxbos i really wanted to love this show . i truly , honestly did . \n \n xxmaj for the first time , gay viewers get their own version of the "" xxmaj the xxmaj bachelor "" . xxmaj with the help of his obligatory "" hag "" xxmaj xxunk , xxmaj james , a good looking , well - to - do thirty - something has the chance",negative


#### Train the last layer of the classifier

In [20]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.577596,0.606892,0.711443,00:03


/home/molly/miniconda3/envs/fastai1-8/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


#### Unfreeze the classifier model and fine-tune

In [21]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.450506,0.459237,0.800995,00:05
1,0.388345,0.387783,0.845771,00:05
2,0.319173,0.357060,0.850746,00:05


/home/molly/miniconda3/envs/fastai1-8/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/home/molly/miniconda3/envs/fastai1-8/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant t

#### Test the classifier
We can use our model to predict on a few example of movie review-like raw text by using the [`Learner.predict`](/basic_train.html#Learner.predict) method. 

Our model is 70% sure that this is a positive review.

In [22]:
learn.predict('Although there was lots of blood and violence, I did not think this film was scary enough.')

(Category tensor(0), tensor(0), tensor([0.6025, 0.3975]))

Our model is 83% sure that this is a positive review:

In [23]:
learn.predict('Not so good World War II epic film')

(Category tensor(1), tensor(1), tensor([0.1317, 0.8683]))

Bottom line: the model did not do a good job, misclassifying both reviews (with high confidence, to boot!)

#### Let's train some more, and re-try these examples to see if we can do better

In [30]:
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-5, 1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.122418,0.401690,0.850746,00:05
1,0.110225,0.397126,0.845771,00:05
2,0.096600,0.419464,0.845771,00:05
3,0.089299,0.437771,0.845771,00:05
4,0.076575,0.436292,0.850746,00:05


/home/molly/miniconda3/envs/fastai1-8/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/home/molly/miniconda3/envs/fastai1-8/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant t

#### Re-try the above examples

In [31]:
learn.predict('Although there was lots of blood and violence, I did not think this film was scary enough.')

(Category tensor(0), tensor(0), tensor([0.9625, 0.0375]))

In [32]:
learn.predict('Not so good World War II epic film')

(Category tensor(0), tensor(0), tensor([0.5441, 0.4559]))

The model seems to have benefitted from the extra training, since it now correctly classifies both reviews as `negative`, with high confidence.

#### Make up your own text in the style of a movie review. Then use our classifier to see what it thinks!

In [41]:
learn.data.train

AttributeError: train

In [33]:
learn.opt, lamb, large bs optimizer, bs = 256,512,1024,2048,4096

OptimWrapper over Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.95, 0.99)
    eps: 1e-08
    lr: 4.0000000000000004e-11
    weight_decay: 0

Parameter Group 1
    amsgrad: False
    betas: (0.95, 0.99)
    eps: 1e-08
    lr: 4.0000000000000004e-11
    weight_decay: 0

Parameter Group 2
    amsgrad: False
    betas: (0.95, 0.99)
    eps: 1e-08
    lr: 1.2649110640673518e-10
    weight_decay: 0

Parameter Group 3
    amsgrad: False
    betas: (0.95, 0.99)
    eps: 1e-08
    lr: 1.2649110640673518e-10
    weight_decay: 0

Parameter Group 4
    amsgrad: False
    betas: (0.95, 0.99)
    eps: 1e-08
    lr: 4.000000000000001e-10
    weight_decay: 0

Parameter Group 5
    amsgrad: False
    betas: (0.95, 0.99)
    eps: 1e-08
    lr: 4.000000000000001e-10
    weight_decay: 0

Parameter Group 6
    amsgrad: False
    betas: (0.95, 0.99)
    eps: 1e-08
    lr: 1.2649110640673522e-09
    weight_decay: 0

Parameter Group 7
    amsgrad: False
    betas: (0.95, 0.99)
    eps: 1e-08
    l